In [48]:
class Operation():
    def __init__(self,input_nodes=[]):
        self.input_nodes=input_nodes
        self.output_nodes=[]
        
        for node in input_nodes:
            node.output_nodes.append(self)
            
        _default_graph.operations.append(self)
        
        
        def compute():
            pass


In [49]:
class add(Operation):
    def __init__(self, x,y):
        super().__init__([x,y])
        
    def compute(self,x,y):
        self.inputs=[x,y]
        return x+y
        

In [50]:
class multiply(Operation):
    def __init__(self, x,y):
        super().__init__([x,y])
        
    def compute(self,x,y):
        self.inputs=[x,y]
        return x*y

In [51]:
class matmul(Operation):
    def __init__(self,x,y):
        super().__init__([x,y])
    
    def compute(self,x,y):
        self.inputs=[x,y]
        return x.dot(y)

In [72]:
class sigmoid(Operation):
    def __init__(self,x):
        super().__init([x])
        
    def compute(self,x):
        self.inputs=[x]
        return 1/(1+np.exp(-x))

In [52]:
class Placeholder():
    
    def __init__(self):
        self.output_nodes=[]
        _default_graph.placeholders.append(self)

In [53]:
class Variable():
    def __init__(self,initial_value=None):
        
        self.value= initial_value
        self.output_nodes = []
        _default_graph.variables.append(self)
    

In [54]:
class Graph():
    
    def __init__(self):
        self.operations =[]
        self.variables =[]
        self.placeholders = []
        
    def set_as_default(self):
        global _default_graph
        _default_graph = self
        
    

In [56]:
g = Graph()

In [57]:
g.set_as_default()

In [59]:
def traverse_postorder(operation):
    
    nodes_postorder=[]
    def recurse(node):
        if isinstance(node,Operation):
            for input_node in node.input_nodes:
                recurse(input_node)
        nodes_postorder.append(node)
            
            
    recurse(operation)
    return nodes_postorder

In [60]:
class Session():
    def run(self,operation,feed_dict={}):
        
        nodes_postorder = traverse_postorder(operation)
        
        for node in nodes_postorder:
            if type(node)==Placeholder:
                node.output= feed_dict[node]
            elif type(node)==Variable:
                node.output = node.value
            
            else:
                node.inputs= [input_node.output for input_node in node.input_nodes]
                
                node.output = node.compute(*node.inputs)
                
            if type(node.output)==list:
                node.output= np.array(node.output)
            
        return operation.output
                
        

In [68]:
A = Variable(4)
B = Variable(2)
X = Placeholder()
y = multiply(A,X)
z= add(y,B)

In [69]:
sess =Session()
p=sess.run(z,feed_dict={X:10})

In [70]:
y

In [71]:
p

42